In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import networkx as nx
from geopy.distance import geodesic
import numpy

small_path = "./main_data/small_data/"
main_path = "./main_data/"
result_path = "./results/"

**Load Data**

In [2]:
# Function -------------------------------------------------------------
def load_total_demand():

    yearly_sales = pd.read_csv(f'{main_path}vw_yearly_sales.csv')
    return yearly_sales


# Function -------------------------------------------------------------
def load_dcs(small = False):
  
    # NOTE: stateconst is a constant used to account for the different prices for the cost of living and rent in various states
    if small:
        dcs = pd.read_csv(f'{small_path}small_list_of_dcs.csv', encoding= 'unicode_escape', names=['ID', "Name","City","State","Address Line 1","Address Line 2", "Coordinates", "Area m3", "Facility Type", "StateConst"])
    else:
        dcs = pd.read_csv(f'{main_path}list_of_dcs.csv', encoding= 'unicode_escape', names=['ID', "Name","City","State","Address Line 1","Address Line 2", "Coordinates", "Area m3", "Facility Type", "StateConst"])

    dcs['Area sqft'] = round(dcs['Area m3'] * 10.7639)
    return dcs


# Function -------------------------------------------------------------
def load_dealers(small = False):
  # https://www.vw.com/app/dccsearch/vw-us/en/Find%20a%20Volkswagen%20Dealer/+/38.353354499999995/-95.3817145/3/+/+/+/+

    if small:
        dealers = pd.read_csv(f'{small_path}small_list_of_dealers.csv',names=['ID', "Name","Address Line 1","Address Line 2", "Website","Phone Number", "Coordinates"])
    else:
        # FULL DATASET
        dealers= pd.read_csv(f'{main_path}list_of_dealers.csv',names=['ID', "Name","Address Line 1","Address Line 2", "Website","Phone Number", "Coordinates"])
    
    return dealers

# Function -------------------------------------------------------------
def load_plants():
    
    plants= pd.read_csv(f'{main_path}list_of_vw_plants_and_products.csv', encoding= 'unicode_escape',names=['ID', "City","Country", "Model","Coordinates","Current/Former plant"])
    plants_usa = plants.loc[plants["Country"]=="USA"]

    return plants_usa


# Function -------------------------------------------------------------
def load_dealer_demands():

    def single_sample():
        # 4 samples
        dealers= pd.read_csv(f'{small_path}small_list_of_demand_data.csv',names=['ID',"Name","Rating","Proportion","na"])
        del dealers[dealers.columns[-1]]
        return dealers

    def full_data():
        # FULL DATASET
        dealers= pd.read_csv(f'{main_path}list_of_demand_data.csv',names=['ID',"Name","Rating","Proportion","na", "na2", "na3"])
        del dealers[dealers.columns[-1]]
        del dealers[dealers.columns[-1]]
        del dealers[dealers.columns[-1]]
        return dealers

    # either call single_sample or full_data
    dealers = full_data() 

    return dealers

In [3]:
# Get all plants, DCs, and Dealers
plants = load_plants()
dcs = load_dcs()
dealers = load_dealers()

In [4]:
dcs

,ID,Name,City,State,Address Line 1,Address Line 2,Coordinates,Area m3,Facility Type,StateConst,Area sqft
0,0,Volkswagen Parts Distribution Center,Ontario,CA,3095 E Cedar St,"Ontario, CA 91761","34.0376080290585, -117.58641507359258",30456,1,68814,327825.0
1,1,Volkswagen Parts Distribution Center,Rocklin,CA,1100 Tinker Rd,"Rocklin, CA 95765, United States","38.81328171265908, -121.30150120229099",18360,1,91290,197625.0
2,2,Volkswagen Parts Distribution Center,Jacksonville,FL,4150 Perimeter Industrial Pkwy N,"Jacksonville, FL 32219, United States","30.373771993405768, -81.75890718718041",362670,2,82698,3903744.0
3,3,Volkswagen Parts Distribution Center,Cranbury,NJ,47 Station Rd,"East Windsor, NJ 08512, United States","40.3071068545627, -74.50867073108051",103200,2,77204,1110834.0
4,4,Volkswagen Regional Distribution Center,Lenoir City,TN,2560 Buttermilk Rd W,"Lenoir City, TN 37771, United States","35.88214676007374, -84.3612681600479",45000,1,66620,484376.0
5,5,Volkswagen Group of America Parts Distribution...,Ft. Worth,TX,230 Intermodal Pkwy,"Fort Worth, TX 76177, United States","32.96678488424622, -97.32814574044846",55954,1,62672,602283.0
6,6,Volkswagen Parts Distribution Center,Pleasant Prairie,WI,11589 88th Ave,"Pleasant Prairie, WI 53158, United States","42.5108270307348, -87.91110551996559",23217,1,66901,249905.0


**Location Work**

In [5]:
# NOTE: Distances are in miles (mi).
# NOTE: All areas are in square-feet (sqft)

# Function -------------------------------------------------------------
def dc_to_dealer_distances(dcs, dealers): 
    
    # using geodesic distance, may want to add 15% or something onto distance
    distance_dc_to_dealer = numpy.zeros((len(dcs), len(dealers)))

    for dc_id, dc in dcs.iterrows():

      for dealer_id, dealer in dealers.iterrows():

        point1, point2 = dc["Coordinates"].split(","), dealer["Coordinates"].split(",")
        distance = geodesic(point1, point2).miles
        dc_id = dc["ID"]
        dealer_id = dealer["ID"]
        distance_dc_to_dealer[dc_id][dealer_id] = distance

    distance_dc_to_dealer_df = pd.DataFrame(distance_dc_to_dealer)
    
    return distance_dc_to_dealer_df


# Function -------------------------------------------------------------
def plant_to_dc_distances(plants, dcs):
  
  # get coordinates of the 1 usa plant
  plant_coordinates = plants['Coordinates'].iloc[0].split(",")
  distance_plants_to_dcs = [] # since we only have 1 usa plant we can just make everything from there (very convenient)
  
  for dcs_id, dc in dcs.iterrows():

    point1 = plant_coordinates
    point2 = dc["Coordinates"].split(",")
    distance = geodesic(point1, point2).miles
    dc_id = dc["ID"]
    distance_plants_to_dcs.append(distance)
  
  return distance_plants_to_dcs

# Get distances from plant-to-DC and DC-to-dealer
dcs_to_dealers = dc_to_dealer_distances(dcs, dealers)
plants_to_dcs = plant_to_dc_distances(plants, dcs)

**Calculate Demand**

In [6]:
# Function -------------------------------------------------------------
def get_dealer_demands(year):
  
  dealer_demands = load_dealer_demands()
  # demand for each specific car type
  total_demands = load_total_demand() 

  def get_demands(year):
    year_demands = total_demands.loc[total_demands['year'] == year]
    year_demands = year_demands[["Atlas","Cross Sport","ID4","Passat"]]
    return year_demands

  year_demands = get_demands(2021)
  year_demands = year_demands.values.tolist()[0]

  proportion_sum = dealer_demands["Proportion"].sum()

  dealer_demands [["Ratio of sales"]] = None
  dealer_demands["Ratio of sales"] = dealer_demands["Proportion"]/proportion_sum

  temp = []

  for x in list(dealer_demands["Ratio of sales"]):

    temp1=[]

    for y in list(year_demands):
      
      temp1.append(round(x*y))

    temp.append(temp1)

  sales = pd.DataFrame(temp, columns=["Atlas","Cross Sport","ID4","Passat"])
  
  return sales

**Calculating Costs**

In [7]:
# Function -------------------------------------------------------------
def calculate_dc_cost():
    
    # Function -------------------------------------------------------------
    def calculate_dc_cost_monthly(id, state, facility_type, areasqft):
        
        #2$ baseline
        # cost = 1.5*areasqft 

        # https://glcdistribution.com/customer-resources/tools/warehousing-insurance-calculator/ 1 million dollar standard
        insurance = 3850 

        # https://www.payscale.com/research/CA/Job=Warehouse_Material_Handler/Hourly_Rate
        employee_salaries = (areasqft/5000)*20*(40*4) 

        # pay 0.25 cents per sqft for utility
        utilities = 0.25*areasqft

        # $600 – $950 per month to lease a $50,000 forklift.
        # https://discord.com/channels/260272353118912522/1093655130076893284/1129635416409133076 1 forklift services 5000 sqft
        # We want to overwork it.
        forklift_cost =  areasqft/10000  * 600

        # https://discord.com/channels/260272353118912522/1093655130076893284/1129639090942398504
        price_it_costs_to_build_warehouse_sqft = 100*areasqft
        property_tax = 6/1000*price_it_costs_to_build_warehouse_sqft

        # 1000 fixed cost monitoring + 20$/h * 40h/week * 28 people
        security = 1000 + (5 * 8 * 28*20) 

        # Assuming Facility 1 has much lower space and so should be priced lower.
        if facility_type == 1:
            ft = 0.10
        else:
            ft = 0.5

        # 508 is the price of 100 big macs and state is the price of 100 big macs in that state
        price = (insurance + employee_salaries + utilities + forklift_cost + security + property_tax)*((state*0.1)/508)*ft
        
        return price

    prices = []

    for i in range(len(dcs.index)):
        moneys = calculate_dc_cost_monthly(float(dcs.loc[i,["ID"]]), float(dcs.loc[i,["StateConst"]]), float(dcs.loc[i,["Facility Type"]]),float(dcs.loc[i,["Area sqft"]]))
        prices.append(round(moneys*12, 2))
    
    return prices

# Function -------------------------------------------------------------
def calculate_dc_shipment_capacity(capacity = False):
  
  # This is: C_i
  # The largest car to consider is the Cross Sport
  # It has dimensions: 4,966 mm L x 1,990 mm W x 1,723 mm H
  # we can round this up to: 17 ft length, 7 foot wide, 6 foot tall
  # add about 2 ft to everything as a buffer
  # new measurement is about 18 ft length, 8ft wide, 8 foot tall
  # so, we can predict that 2 car takes up 144/2, each car takes 72 sq foot

  # from https://discord.com/channels/260272353118912522/1093655130076893284/1131271495004459018,
  # we see that 60-80% of DCs are for storage. Let's assume 60%
  vehicle_capacity = dcs["Area sqft"]*0.6 / 72

  # from https://discord.com/channels/260272353118912522/1093655130076893284/1131266537869820075
  # we see that most vehicles stay in the dc a couple of weeks (take 2 weeks)
  # we can frame this as: All vehicles in the DCS arrive at time 0, and after 2 weeks all are released
  annual_throughput = round(vehicle_capacity*6, 0)

  if capacity:
    return list(vehicle_capacity)
  else:
    return (list(annual_throughput))
  

# Function -------------------------------------------------------------
def calculate_hold_costs():
    # We will assume the holding costs are the total number of vehicles in the warehouse divided
    # by the total costs of the warehouse.
    # This will be multiplied by a cost factor to account for large costs with lower capacities.
    dc_cost = calculate_dc_cost()
    total_vehicles = calculate_dc_shipment_capacity(True)
    dc_type = dcs[["Facility Type"]].to_numpy()

    holding_costs = []

    for i in range(0, len(dcs.index)):

        if dc_type[i,] == 1:
            holding = round(total_vehicles[i]/dc_cost[i]*10000, 2)
        else:
            holding = round(total_vehicles[i]/dc_cost[i]*50000, 2)

        holding_costs.append(holding)

    return holding_costs

In [8]:
calculate_dc_shipment_capacity()

[16391.0, 9881.0, 195187.0, 55542.0, 24219.0, 30114.0, 12495.0]

In [9]:
calculate_dc_cost()

[8702729.13,
 7193265.63,
 593672649.84,
 159487904.66,
 12243908.3,
 14223908.57,
 6552127.22]

In [10]:
calculate_hold_costs()

[3.14, 2.29, 2.74, 2.9, 3.3, 3.53, 3.18]

**Determining the Cost of Transportation**

In [11]:
## THE FUNCTION OF SHIPPING TRUCKS CAN BE APPROXIMATED BY A PIECEWISE FUNCTION
# https://www.desmos.com/calculator/ipxbog2uyr
# constructed from the data here: https://www.marketwatch.com/guides/car-shipping/truck-shipping/#:~:text=According%20to%20quotes%20we%20received,when%20using%20an%20enclosed%20carrier.

# Function -------------------------------------------------------------
def fixed_cost_shipment(): 
  # This is: T_1
  # cost to drive 0 miles
  return 186 


# Function -------------------------------------------------------------
def calculate_variable_cost_shipment(mileage): 
  # This is: T_2
  # if mileage < 1135:
  #   return mileage * (551/535)
  # return mileage * (0.43)
  return (0.43)

  # TODO: WE ARE ASSUMING FOR SIMPLICYT SAKE 0.43$/MILE AND ARE NOT CONSIDERING THE OTHER FUNCTION FOR MODELLING RIGHT NOW

**DC Inventory Carrying Costs**

**Time of Model Building**

In [12]:
## The modelling?
import gurobipy as gp
from gurobipy import GRB

# Create environment with WLS license
e = gp.Env(empty=True)
e.setParam('WLSACCESSID', '47fa1c45-5377-4f93-87d4-3da7da2b6955')
e.setParam('WLSSECRET', '7dfbddcb-7333-4ec7-927d-b459cffd9565')
e.setParam('LICENSEID', 868415)
e.start()

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 868415
Academic license - for non-commercial use only - registered to anita.yang@uwaterloo.ca


<gurobipy.Env, Parameter changes: WLSAccessID=(user-defined), WLSSecret=(user-defined), LicenseID=868415>

**Define Model Sets**

In [13]:
# Get all distances, demands, and costs
dcs_to_dealers = dc_to_dealer_distances(dcs, dealers)
plants_to_dcs = plant_to_dc_distances(plants, dcs)
dealer_demands = get_dealer_demands(2021) ## car demand for each dealership
dc_annual_cost = calculate_dc_cost()
holding_costs = calculate_hold_costs()

**Define Parameters**

In [14]:
# define sets
# set S
plants_set = [0]
# set I
dcs_set = list(dcs["ID"])
# set J
dealers_set = list(dealers["ID"])
# we define 0 to 3, corresponding to Atlas, CrossSport, ID4 and Passat vehicles (set V)
vehicle_set = [0,1,2,3]
# set P
facility_type_set = [0, 1]

In [15]:
# Parameter prep - we change the dataframes to arrays of sorts, which we can use indexes and such in the formulation
d_jv = dealer_demands.values
m_ij = dcs_to_dealers.values
c_si = plants_to_dcs
# load factor is 1
L_j = 1 
T_1 = fixed_cost_shipment()
T_2 = calculate_variable_cost_shipment("dummy_value") # see code comments on why we do this as a temp hack for a preliminary model
f_ip = dc_annual_cost
C_i = calculate_dc_shipment_capacity()

In [16]:
# Create Model
m = gp.Model(env=e)

# CREATE THE DECISION VARIABLES HERE
x1_siv = m.addVars(plants_set, dcs_set, vehicle_set, vtype=GRB.INTEGER)
x2_ijv = m.addVars(dcs_set, dealers_set, vehicle_set, vtype=GRB.INTEGER )
y_ip = m.addVars(dcs_set, facility_type_set, vtype=GRB.BINARY)

**Objective Function**

In [17]:

obj1 = sum((x2_ijv[i, j, v]/L_j)*(T_1 + T_2*m_ij[i, j]) for v in vehicle_set for j in dealers_set for i in dcs_set)

obj2 = sum(sum(x2_ijv[i, j, v] + x1_siv[s, i, v] for v in vehicle_set for j in dealers_set for s in plants_set) * c_si[i] for i in dcs_set)

obj3 = sum(sum(x1_siv[s, i, v] for v in vehicle_set for s in plants_set) * holding_costs[i] for i in dcs_set)

obj4 = sum(f_ip[i]*y_ip[i, p] for p in facility_type_set for i in dcs_set)

m.setObjective(obj1 + obj2 + obj3 + obj4, gp.GRB.MINIMIZE)

**Constraints**

In [18]:
# Constraint 1: Demand at each market area for each vehicle type must be met
m.addConstrs(sum(x2_ijv[i, j, v] for i in dcs_set) == d_jv[j, v] for v in vehicle_set for j in dealers_set)

# Constraint 2: Vehicle flows between plants and DCs must be conserved
m.addConstrs(sum(x1_siv[s, i, v] for s in plants_set) == sum(x2_ijv[i, j, v] for j in dealers_set) for i in dcs_set for v in vehicle_set)

# Constraint 3 min: Total vehicle flow to each DC must satisfy the minimum capacity requirement
m.addConstrs(
    (C_i[i] * y_ip[i, 1] <= sum(x2_ijv[i, j, v] for v in vehicle_set for j in dealers_set) for i in dcs_set),
    name="constraint_3_min"
)

# Constraint 3 max: Total vehicle flow to each DC must satisfy the maximum capacity requirement
m.addConstrs(
    (sum(x2_ijv[i, j, v] for v in vehicle_set for j in dealers_set) <= C_i[i] * y_ip[i, 0] + sum(d_jv[j, v] for v in vehicle_set for j in dealers_set) * y_ip[i, 1] for i in dcs_set),
    name="constraint_3_max"
)

# # Constraint 4: Shipment quantities must be nonnegative
m.addConstrs(x1_siv[s, i, v] >= 0 for s in plants_set for v in vehicle_set for i in dcs_set)
m.addConstrs(x2_ijv[i, j, v] >= 0 for i in dcs_set for v in vehicle_set for j in dealers_set)

# Constraint 6: The distribution centers must be selected so that all market areas can be reached within r days. 
#               Suppose a truck travels on average 300 miles a day. (Redundant constraint)
r=60
m.addConstrs((sum(x2_ijv[i, j, v] for v in vehicle_set) == 0 for i in dcs_set for j in dealers_set if m_ij[i][j] / 300 > r))

{}

**Optimize**

In [19]:
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11390H @ 3.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license - for non-commercial use only - registered to anita.yang@uwaterloo.ca
Optimize a model with 20486 rows, 17906 columns and 89397 nonzeros
Model fingerprint: 0x14ac9650
Variable types: 0 continuous, 17906 integer (14 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+05]
  Objective range  [3e+02, 6e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+02]
Found heuristic solution: objective 7.609342e+10
Presolve removed 17923 rows and 43 columns
Presolve time: 0.06s
Presolved: 2563 rows, 17863 columns, 51019 nonzeros
Variable types: 0 continuous, 17863 integer (90 binary)
Found heuristic solution: objective 7.538026e+10

Root relaxation: objective 7.158113e+09, 2551 iterations, 0.03 seconds (0.00 work units)

    No

In [ ]:
x1_siv

In [20]:
y_ip

{(0, 0): <gurobi.Var C17892 (value -0.0)>,
 (0, 1): <gurobi.Var C17893 (value -0.0)>,
 (1, 0): <gurobi.Var C17894 (value -0.0)>,
 (1, 1): <gurobi.Var C17895 (value -0.0)>,
 (2, 0): <gurobi.Var C17896 (value -0.0)>,
 (2, 1): <gurobi.Var C17897 (value 0.0)>,
 (3, 0): <gurobi.Var C17898 (value -0.0)>,
 (3, 1): <gurobi.Var C17899 (value -0.0)>,
 (4, 0): <gurobi.Var C17900 (value -0.0)>,
 (4, 1): <gurobi.Var C17901 (value 1.0)>,
 (5, 0): <gurobi.Var C17902 (value -0.0)>,
 (5, 1): <gurobi.Var C17903 (value -0.0)>,
 (6, 0): <gurobi.Var C17904 (value -0.0)>,
 (6, 1): <gurobi.Var C17905 (value -0.0)>}